In [ ]:
import tensorflow as tf
print(tf.__version__)


2.19.0


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from google.colab import drive
from PIL import Image
import time

# ========== SETUP ========== #
# Mount Google Drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

# Configuration
dataset_path = "/content/drive/MyDrive/dfu /PartB_DFU_Dataset/Infection/Aug-Negative"  # UPDATE THIS
save_path = "/content/drive/MyDrive/ViTGAN_outputs"         # UPDATE THIS
os.makedirs(save_path, exist_ok=True)

# Hyperparameters
img_size = 64
channels = 3
latent_dim = 128
batch_size = 64
epochs = 500
save_interval = 25

# ========== DATA PIPELINE ========== #
def load_dataset(dataset_path, img_size):
    """Optimized data loading pipeline"""
    image_paths = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path)
                  if f.endswith(('.jpg', '.jpeg', '.png'))]

    def preprocess_image(img_path):
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [img_size, img_size])
        img = (img - 127.5) / 127.5  # Normalize to [-1, 1]
        return img

    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(2000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

dataset = load_dataset(dataset_path, img_size)

# ========== MODEL ARCHITECTURE ========== #
def build_generator(latent_dim):
    inputs = layers.Input(shape=(latent_dim,))

    # Foundation
    x = layers.Dense(8*8*256)(inputs)
    x = layers.Reshape((8, 8, 256))(x)

    # Upsampling blocks
    x = layers.Conv2DTranspose(128, 4, 2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2DTranspose(64, 4, 2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)

    # Output (ensure exact dimensions)
    x = layers.Conv2DTranspose(channels, 4, padding='same', activation='tanh')(x)
    x = layers.Cropping2D(((0,1),(0,1)))(x) if img_size % 2 != 0 else x
    x = layers.Resizing(img_size, img_size)(x)

    return keras.Model(inputs, x, name="generator")

def build_discriminator():
    inputs = layers.Input(shape=(img_size, img_size, channels))

    # Downsampling blocks
    x = layers.Conv2D(64, 4, 2, padding='same')(inputs)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(128, 4, 2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)

    # Classification
    x = layers.Flatten()(x)
    x = layers.Dense(1)(x)

    return keras.Model(inputs, x, name="discriminator")

# ========== TRAINING SETUP ========== #
# Initialize models
generator = build_generator(latent_dim)
discriminator = build_discriminator()

# Optimizers
g_optimizer = Adam(0.0002, beta_1=0.5)
d_optimizer = Adam(0.0002, beta_1=0.5)

# Loss function
cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)

# ========== TRAINING LOOP ========== #
@tf.function
def train_step(real_images):
    # Train discriminator
    noise = tf.random.normal([batch_size, latent_dim])
    with tf.GradientTape() as d_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(real_images, training=True)
        fake_output = discriminator(generated_images, training=True)
        d_loss = (cross_entropy(tf.ones_like(real_output), real_output) +
                 cross_entropy(tf.zeros_like(fake_output), fake_output)) / 2

    d_gradients = d_tape.gradient(d_loss, discriminator.trainable_variables)
    d_optimizer.apply_gradients(zip(d_gradients, discriminator.trainable_variables))

    # Train generator
    noise = tf.random.normal([batch_size, latent_dim])
    with tf.GradientTape() as g_tape:
        generated_images = generator(noise, training=True)
        fake_output = discriminator(generated_images, training=True)
        g_loss = cross_entropy(tf.ones_like(fake_output), fake_output)

    g_gradients = g_tape.gradient(g_loss, generator.trainable_variables)
    g_optimizer.apply_gradients(zip(g_gradients, generator.trainable_variables))

    return d_loss, g_loss

def save_images(epoch):
    """Save grid and individual images to Drive"""
    os.makedirs(save_path, exist_ok=True)
    noise = tf.random.normal([25, latent_dim])
    images = generator(noise, training=False)
    images = (images * 127.5 + 127.5).numpy().astype('uint8')

    # Save grid
    plt.figure(figsize=(10,10))
    for i in range(25):
        plt.subplot(5, 5, i+1)
        plt.imshow(images[i])
        plt.axis('off')
    plt.savefig(f"{save_path}/grid_{epoch:04d}.png")
    plt.close()

    # Save individual images
    for i, img in enumerate(images):
        Image.fromarray(img).save(f"{save_path}/img_{epoch:04d}_{i:02d}.png")

    print(f"Saved outputs to {save_path}")

def train(dataset, epochs, save_interval):
    for epoch in range(epochs):
        start = time.time()

        total_d_loss = 0
        total_g_loss = 0
        num_batches = 0

        for batch in dataset:
            d_loss, g_loss = train_step(batch)
            total_d_loss += d_loss
            total_g_loss += g_loss
            num_batches += 1

        # Save progress
        if epoch % save_interval == 0 or epoch == epochs - 1:
            save_images(epoch)

        print(f"⏱️ Epoch {epoch+1}/{epochs} | "
              f"Disc Loss: {total_d_loss/num_batches:.4f} | "
              f"Gen Loss: {total_g_loss/num_batches:.4f} | "
              f"Time: {time.time()-start:.2f}s")

# ========== START TRAINING ========== #
print(" Training started - monitoring progress...")
train(dataset, epochs, save_interval)

# Final save
generator.save(f"{save_path}/final_generator.h5")
print(f" Training complete! Models saved to {save_path}")

Mounted at /content/drive
 Training started - monitoring progress...
Saved outputs to /content/drive/MyDrive/ViTGAN_outputs
⏱️ Epoch 1/500 | Disc Loss: 0.5485 | Gen Loss: 1.4087 | Time: 156.49s
⏱️ Epoch 2/500 | Disc Loss: 0.4577 | Gen Loss: 1.7936 | Time: 94.88s
⏱️ Epoch 3/500 | Disc Loss: 0.6580 | Gen Loss: 0.8940 | Time: 88.30s
⏱️ Epoch 4/500 | Disc Loss: 0.6929 | Gen Loss: 0.6239 | Time: 87.11s
⏱️ Epoch 5/500 | Disc Loss: 0.7062 | Gen Loss: 0.7558 | Time: 86.50s
⏱️ Epoch 6/500 | Disc Loss: 0.7025 | Gen Loss: 0.7501 | Time: 85.90s
⏱️ Epoch 7/500 | Disc Loss: 0.7048 | Gen Loss: 0.7201 | Time: 87.00s
⏱️ Epoch 8/500 | Disc Loss: 0.6969 | Gen Loss: 0.7163 | Time: 85.90s
⏱️ Epoch 9/500 | Disc Loss: 0.6929 | Gen Loss: 0.7224 | Time: 85.90s
⏱️ Epoch 10/500 | Disc Loss: 0.6999 | Gen Loss: 0.7281 | Time: 85.90s
⏱️ Epoch 11/500 | Disc Loss: 0.6908 | Gen Loss: 0.7304 | Time: 85.90s
⏱️ Epoch 12/500 | Disc Loss: 0.6966 | Gen Loss: 0.6928 | Time: 86.80s
⏱️ Epoch 13/500 | Disc Loss: 0.6885 | Gen Lo

KeyboardInterrupt: 

In [ ]:
!pip install tensorflow-hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.0 MB/s eta 0:00:00


In [ ]:
!pip install opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 19.3 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import numpy as np

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)  # Load image
        if img is not None:
            img = cv2.resize(img, (64, 64))  # Resize if needed
            images.append(img)
    return np.array(images)

real_images_path = "/content/drive/MyDrive/dfu /PartB_DFU_Dataset/Infection/Aug-Negative" # Update this!
real_images = load_images_from_folder(real_images_path)

print(f"Loaded {len(real_images)} real images")



Loaded 2946 real images


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from scipy.linalg import sqrtm

# ✅ Load images (Optimized for Speed)
def load_images(folder, max_images=1200, size=(64, 64)):
    images = []
    count = 0
    for filename in os.listdir(folder):
        if count >= max_images:
            break
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images.append(img)
            count += 1
    return np.array(images, dtype=np.float32) / 255.0  # Normalize to [0,1]

# Paths
real_path = "/content/drive/MyDrive/dfu /PartB_DFU_Dataset/Infection/Aug-Positive"
gen_path = "/content/drive/MyDrive/ViTGAN_outputs"
# Load Images
real_imgs = load_images(real_path, 1200)
gen_imgs = load_images(gen_path, 1200)

# ✅ Ensure Same Number of Images
num_imgs = min(len(real_imgs), len(gen_imgs))
real_imgs, gen_imgs = real_imgs[:num_imgs], gen_imgs[:num_imgs]

# ✅ FAST SSIM Computation
def compute_ssim_batch(real, generated):
    return np.mean([
        ssim(real[i], generated[i], data_range=1.0, channel_axis=2)
        for i in range(len(real))
    ])

# ✅ FAST PSNR Computation
def compute_psnr_batch(real, generated):
    mse = np.mean((real - generated) ** 2, axis=(1, 2, 3))
    psnr = 20 * np.log10(1.0 / np.sqrt(mse))
    return np.mean(psnr)

# ✅ FAST FID Computation with EfficientNetB0
def compute_fid(real, generated):
    model = EfficientNetB0(include_top=False, pooling='avg', input_shape=(64, 64, 3))

    # Extract Features in One Batch for Speed
    real_feat = model.predict(preprocess_input(real * 255), batch_size=32)
    gen_feat = model.predict(preprocess_input(generated * 255), batch_size=32)

    # Compute Mean and Covariance
    mu_r, sigma_r = real_feat.mean(axis=0), np.cov(real_feat, rowvar=False)
    mu_g, sigma_g = gen_feat.mean(axis=0), np.cov(gen_feat, rowvar=False)

    diff = mu_r - mu_g
    cov_mean = sqrtm(sigma_r @ sigma_g)

    if np.iscomplexobj(cov_mean):
        cov_mean = cov_mean.real  # Avoid Complex Values

    return np.sum(diff**2) + np.trace(sigma_r + sigma_g - 2 * cov_mean)

# ✅ Compute Metrics
ssim_score = compute_ssim_batch(real_imgs, gen_imgs)
psnr_score = compute_psnr_batch(real_imgs, gen_imgs)
fid_score = compute_fid(real_imgs, gen_imgs)

# ✅ Display Results
print(f"SSIM: {ssim_score:.4f} (Higher is Better)")
print(f"PSNR: {psnr_score:.2f} dB (Higher is Better)")
print(f"FID: {fid_score:.2f} (Lower is Better)")


18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 291ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 185ms/step
SSIM: 0.2361 (Higher is Better)
PSNR: 11.90 dB (Higher is Better)
FID: 133.17 (Lower is Better)


In [ ]:
pip install opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 19.7 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.1 MB/s eta 0:00:00


In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input as preprocess_efficientnet
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_inception


In [ ]:
pip install --upgrade tensorflow keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 82.1 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Successfully uninstalled keras-3.8.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolve

In [ ]:
pip install opencv-python numpy tensorflow scikit-image scipy scikit-learn


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input as preprocess_efficientnet
from scipy.linalg import sqrtm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# ✅ Load images efficiently
def load_images(folder, max_images=1200, size=(64, 64)):
    images = []
    count = 0
    for filename in os.listdir(folder):
        if count >= max_images:
            break
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images.append(img)
            count += 1
    return np.array(images, dtype=np.float32) / 255.0  # Normalize to [0,1]

# Paths
real_path = "/content/drive/MyDrive/dfu /PartB_DFU_Dataset/Infection/Aug-Positive"
gen_path = "/content/drive/MyDrive/ViTGAN_outputs"

# Load Images
real_imgs = load_images(real_path, 1200)
gen_imgs = load_images(gen_path, 1200)

# ✅ Ensure same number of images
num_imgs = min(len(real_imgs), len(gen_imgs))
real_imgs, gen_imgs = real_imgs[:num_imgs], gen_imgs[:num_imgs]

# ✅ SSIM Computation
def compute_ssim_batch(real, generated):
    return np.mean([
        ssim(real[i], generated[i], data_range=1.0, channel_axis=2)
        for i in range(len(real))
    ])

# ✅ PSNR Computation
def compute_psnr_batch(real, generated):
    mse = np.mean((real - generated) ** 2, axis=(1, 2, 3))
    psnr = 20 * np.log10(1.0 / np.sqrt(mse))
    return np.mean(psnr)

# ✅ FID Computation
def compute_fid(real, generated):
    model = EfficientNetB0(include_top=False, pooling='avg', input_shape=(64, 64, 3))

    # Extract Features in One Batch for Speed
    real_feat = model.predict(preprocess_efficientnet(real * 255), batch_size=32)
    gen_feat = model.predict(preprocess_efficientnet(generated * 255), batch_size=32)

    # Compute Mean and Covariance
    mu_r, sigma_r = real_feat.mean(axis=0), np.cov(real_feat, rowvar=False)
    mu_g, sigma_g = gen_feat.mean(axis=0), np.cov(gen_feat, rowvar=False)

    diff = mu_r - mu_g
    cov_mean = sqrtm(sigma_r @ sigma_g)

    if np.iscomplexobj(cov_mean):
        cov_mean = cov_mean.real  # Avoid Complex Values

    return np.sum(diff**2) + np.trace(sigma_r + sigma_g - 2 * cov_mean)

# ✅ Classification-Based Metrics (Assuming Thresholding)
def compute_classification_metrics(real, generated, threshold=0.5):
    # Convert images to grayscale and compute similarity
    real_gray = np.mean(real, axis=-1).reshape(len(real), -1)
    gen_gray = np.mean(generated, axis=-1).reshape(len(generated), -1)

    # Compute differences and apply threshold
    diffs = np.linalg.norm(real_gray - gen_gray, axis=1)
    preds = (diffs < threshold).astype(int)  # 1 if similar, 0 if different
    labels = np.ones_like(preds)  # Assume all real images are correct class

    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, zero_division=1)
    recall = recall_score(labels, preds, zero_division=1)
    f1 = f1_score(labels, preds, zero_division=1)
    auroc = roc_auc_score(labels, preds)

    tpr = recall  # True Positive Rate (Same as recall)
    fpr = np.sum((preds == 1) & (labels == 0)) / np.sum(labels == 0) if np.sum(labels == 0) > 0 else 0  # False Positive Rate

    return acc, precision, recall, f1, tpr, fpr, auroc

# ✅ Compute Metrics
ssim_score = compute_ssim_batch(real_imgs, gen_imgs)
psnr_score = compute_psnr_batch(real_imgs, gen_imgs)
fid_score = compute_fid(real_imgs, gen_imgs)
acc, precision, recall, f1, tpr, fpr, auroc = compute_classification_metrics(real_imgs, gen_imgs)

# ✅ Display Results
print(f"SSIM: {ssim_score:.4f} (Higher is Better)")
print(f"PSNR: {psnr_score:.2f} dB (Higher is Better)")
print(f"FID: {fid_score:.2f} (Lower is Better)")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall (TPR): {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"False Positive Rate (FPR): {fpr:.4f}")
print(f"AUROC: {auroc:.4f}")


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 402ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
SSIM: 0.2365 (Higher is Better)
PSNR: 11.92 dB (Higher is Better)
FID: 133.17 (Lower is Better)
Accuracy: 0.0000
Precision: 1.0000
Recall (TPR): 0.0000
F1 Score: 0.0000
False Positive Rate (FPR): 0.0000
AUROC: nan


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
